<a href="https://colab.research.google.com/github/EleonoraBartolomucci/Fairness/blob/master/AmazonGetUserData.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip3 install python-dateutil lxml requests selectorlib gender_guesser

     |████████████████████████████████| 389kB 5.8MB/s 


In [ ]:
#IMPORT E FUNZIONI DI SUPPORTO
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
#from clarifai.rest import ClarifaiApp
#from clarifai.rest import ApiError
import numpy as np
import time
import json
import csv
import random
import pandas as pd
import requests
from lxml import html
import shutil
import os
import json
import datetime
import math
import io
import math
from google.colab import files
from selectorlib import Extractor
import requests 
from time import sleep
from dateutil import parser as dateparser
import gender_guesser.detector as gender


# AUTHENTICATE IN GOOGLE DRIVE
def authenticate():
  auth.authenticate_user()
  gauth = GoogleAuth()
  gauth.credentials = GoogleCredentials.get_application_default()
  drive = GoogleDrive(gauth)
  return drive
drive = authenticate()


def create_folder_in_drive(gdrive, folder_name, parent_folder_id):
  folder_metadata = {'title': folder_name,'mimeType': 'application/vnd.google-apps.folder',
                    'parents': [{"kind": "drive#fileLink", "id": parent_folder_id}]
                    }
  folder = gdrive.CreateFile(folder_metadata)
  folder.Upload()
  print(folder)
  # Return folder informations
  print('title: %s, id: %s' % (folder['title'], folder['id']))
  return folder['id']


def drop_unnamed(df):
  cols = [c for c in df.columns if c.lower()[:7] != 'unnamed']
  return df[cols]


def upload_file(filename, folder_id):
  drive = authenticate()
  fileList = drive.ListFile({'q': "'" + folder_id + "' in parents and trashed=false"}).GetList()
  drive_file = drive.CreateFile({'title': filename, 'parents': [{'id': folder_id}]})
  # Check if file already exists in Google Drive (prevents duplicates)
  for file in fileList:
      if file['title'] == filename:  # The file already exists, then overwrite it
          fileID = file['id']
          drive_file = drive.CreateFile({'id': fileID, 'title': filename, 'parents': [{'id': folder_id}]})
  # Upload user picture on Google Drive
  drive_file.SetContentFile(filename)  # path of local file content
  drive_file.Upload()  # Upload the file.
  return drive_file['id']


def downloadUser(business_id, user_id, photo_folder, counter):
    authenticate()
    
    filename = user_id + '.jpg'
    url = 'https://www.yelp.com/user_details?userid=' + user_id
    folder_id = photo_folder

    # CHECK DUPLICATE
    fileList = drive.ListFile({'q': "'" + folder_id + "' in parents and trashed=false"}).GetList()
    exists = False
    # Check if file already exists in Google Drive (prevents duplicates)
    for file in fileList:
        if file['title'] == filename:  # The file already exists
            exists = True
    
    if not exists:
      try:
        # Find user picture from web page
        page_content = requests.get(url)
        page_content.raise_for_status()
        print(page_content)
        tree = html.fromstring(page_content.content)
        if len(tree.xpath('//*[@id="wrap"]/div[2]/div[1]/div/div[2]/div[1]/div/div/div/a/img/@src')) == 0:
          print('PICTURE NOT FOUND of user '+ user_id)
        else: # Picture found
          image_url = tree.xpath('//*[@id="wrap"]/div[2]/div[1]/div/div[2]/div[1]/div/div/div/a/img/@src')[0]

          drive_file = drive.CreateFile({'title': filename, 'parents': [{'id': folder_id}]})
          
          response = requests.get(image_url, stream=True)
          
          # Create a local copy of user picture
          with open(filename, 'wb') as out_file:
              shutil.copyfileobj(response.raw, out_file)
          del response

          # Upload user picture on Google Drive
          drive_file.SetContentFile(filename)
          drive_file.Upload()
          if os.path.exists(filename):
              os.remove(filename)
          else:
              print("The file does not exist")
          print(user_id + ' downloaded')
      except:
        #code = page_content.status_code
        #if code==503:
        #  print("ERROR 503")
        #if code==404:
        #  print("User not found")
        print("Error")

def downloadUserWithProxy(business_id, user_id, photo_folder, counter, ip_addresses):
    authenticate()
    
    filename = user_id + '.jpg'
    url = 'https://www.yelp.com/user_details?userid=' + user_id
    folder_id = photo_folder

    # CHECK DUPLICATE
    fileList = drive.ListFile({'q': "'" + folder_id + "' in parents and trashed=false"}).GetList()
    exists = False
    # Check if file already exists in Google Drive (prevents duplicates)
    for file in fileList:
        if file['title'] == filename:  # The file already exists
            exists = True
    
    if not exists:
      downloaded = False
      while len(ip_addresses) != 0 and not downloaded:
        proxy_index = 0
        proxy = {"http": ip_addresses[proxy_index], "https": ip_addresses[proxy_index]}
        try:
          # Check the proxy address
          #response = requests.get('https://httpbin.org/ip',proxies=proxy, timeout=5)
          #print(response.json())
          
          # Find user picture from web page
          page_content = requests.get(url, proxies=proxy, timeout=10)
          print(page_content)
          if page_content.status_code != 503:
            tree = html.fromstring(page_content.content)
            if len(tree.xpath('//*[@id="wrap"]/div[2]/div[1]/div/div[2]/div[1]/div/div/div/a/img/@src')) == 0:
              print('PICTURE NOT FOUND of user '+ user_id)
            else: # Picture found
              image_url = tree.xpath('//*[@id="wrap"]/div[2]/div[1]/div/div[2]/div[1]/div/div/div/a/img/@src')[0]

              drive_file = drive.CreateFile({'title': filename, 'parents': [{'id': folder_id}]})
              
              response = requests.get(image_url, proxies=proxy, timeout=30, stream=True)
              
              # Create a local copy of user picture
              with open(filename, 'wb') as out_file:
                  shutil.copyfileobj(response.raw, out_file)
              del response

              # Upload user picture on Google Drive
              drive_file.SetContentFile(filename)
              drive_file.Upload()
              if os.path.exists(filename):
                  os.remove(filename)
              else:
                  print("The file does not exist")
              print(user_id + ' downloaded')
              downloaded = True
              ip_addresses.insert(0, ip_addresses.pop(proxy_index))
          else:
            print("Skipping. Connnection error")
            del ip_addresses[proxy_index]
            print("Proxy rimanenti: ")
            print(ip_addresses)
            #restart_runtime()
        except Exception as e:
          print(e)
          print("Skipping. Connnection error")
          del ip_addresses[proxy_index]
          print("Proxy rimanenti: ")
          print(ip_addresses)
      if len(ip_addresses)==0:
        print("Nessun proxy disponibile")


def get_proxies():
    url = 'https://free-proxy-list.net/'
    response = requests.get(url)
    parser = html.fromstring(response.content)
    proxies = []
    for i in parser.xpath('//tbody/tr')[:500]:
        if i.xpath('.//td[7][contains(text(),"yes")]'):
            #Grabbing IP and corresponding PORT
            proxy = ":".join([i.xpath('.//td[1]/text()')[0], i.xpath('.//td[2]/text()')[0]])
            proxies.append(proxy)
    print(proxies)
    proxies.append('18.223.213.237:3838')
    proxies.append('18.223.103.221:3838')
    proxies.append('117.69.230.23:9999')
    with open('http_proxies.txt', 'w') as f:
      for item in proxies:
        f.write("%s\n" % item)
    return proxies

def scrape(e, url):    
  headers = {
      'authority': 'www.amazon.com',
      'pragma': 'no-cache',
      'cache-control': 'no-cache',
      'dnt': '1',
      'upgrade-insecure-requests': '1',
      'user-agent': 'Mozilla/5.0 (X11; CrOS x86_64 8172.45.0) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/51.0.2704.64 Safari/537.36',
      'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
      'sec-fetch-site': 'none',
      'sec-fetch-mode': 'navigate',
      'sec-fetch-dest': 'document',
      'accept-language': 'en-GB,en-US;q=0.9,en;q=0.8',
  }

  # Download the page using requests
  print("Downloading %s"%url)
  r = requests.get(url, headers=headers)
  # Simple check to check if page was blocked (Usually 503)
  if r.status_code > 500:
      if "To discuss automated access to Amazon data please contact" in r.text:
          print("Page %s was blocked by Amazon. Please try using better proxies\n"%url)
      else:
          print("Page %s must have been blocked by Amazon as the status code was %d"%(url,r.status_code))
      return None
  # Pass the HTML of the page and create 
  return e.extract(r.text)

#####################################################################################################
def detect_gender(id, userName):
  d = gender.Detector(case_sensitive=False)
  gendersList = []
  nameList = []

  genderResponse = d.get_gender(userName)

  if genderResponse=="unknown":
    nameList = userName.split(" ")
  print(nameList)
  for nameElement in nameList:
    if (d.get_gender(nameElement)!="unknown") & (d.get_gender(nameElement)!="andy"):
      if (d.get_gender(nameElement)=="mostly_female"):
        gendersList.append('female')
      else:
        if (d.get_gender(nameElement)=="mostly_male"):
          gendersList.append('male')
        else:
          gendersList.append(d.get_gender(nameElement))
  print(gendersList!=0)
  if len(gendersList)!=0:
    male = 0
    female = 0
    for e in gendersList:
      if e=="male":
        male = male + 1
      else:
        female = female + 1
    if male==female:
      genderResponse = gendersList[0]
    else:
      if male>female:
        genderResponse = "male"
      else:
        genderResponse = "female"
  #else:
    #genderResponse == "unknown"
 
  df_result = pd.DataFrame(columns=['user_id', 'age', 'gender', 'ethnicity'])
 
  if genderResponse != "unknown":
    if genderResponse == "mostly_female":
      genderResponse = "female"
    if genderResponse == "mostly_male":
      genderResponse = "male"
    if genderResponse == "andy":
      genderResponse = np.NaN
    new_row = {'user_id':id,
                'age':'-2',
                'gender':genderResponse,
                'ethnicity':np.NaN}
    df_result = df_result.append(new_row, ignore_index=True)
  else:
    #se non ci sono volti, stampa una stringa di default anziché vuoto
    new_row = {'user_id':id,
              'age':'-2',
              'gender':np.NaN,
              'ethnicity':np.NaN}
    df_result = df_result.append(new_row, ignore_index=True)
  return df_result
#####################################################################################################

In [ ]:
#carico i file YML
download = drive.CreateFile({'id': '12DmvHhjrBHI8rolxkkIeLYF6_ZynuUxT'})
download.GetContentFile('AmazonReviews.yml')

download2 = drive.CreateFile({'id': '1PuYdOat85pFtE1XC6__WIxeIe66BQuEp'})
download2.GetContentFile('AmazonProducts.yml')

download3 = drive.CreateFile({'id': '1s2JwxZqTHALk1VKqpe_shwMxLUIhKP-V'})
download3.GetContentFile('CategorySearch.yml')

download4 = drive.CreateFile({'id': '1YKZmpUdqKzCA2xOgYTgRU_QGso3NG1Xx'})
download4.GetContentFile('AmazonUserPage.yml')

# Create Extractors by reading from the YAML file
userExtractor = Extractor.from_yaml_file('AmazonUserPage.yml')
reviewsExtractor = Extractor.from_yaml_file('AmazonReviews.yml')
productExtractor = Extractor.from_yaml_file('AmazonProducts.yml')
categoryExtractor = Extractor.from_yaml_file('CategorySearch.yml') 

In [ ]:
#inizializza queries
download = drive.CreateFile({'id': '1V-L7IqaPVcBp4RvWKnUXt00SHrsT1e9O'})
download.GetContentFile('queries.csv')

queries = pd.read_csv("queries.csv")
queries = drop_unnamed(queries)

for i, x in queries.iterrows():
  if x["products_gtree"]==-1:
    df = pd.DataFrame(columns=["position","asin","url"])
    df.to_csv(str(x["cardinal"])+ "_gtree.csv")
    id_new_folder = create_folder_in_drive(drive, str(x["cardinal"]), '13N3YrLs4LD8i4uZ5JJqrkEVCVUPq1Yje')
    id_new_folder2 = create_folder_in_drive(drive, "reviews_folder", id_new_folder)
    id_new_file = upload_file(str(x["cardinal"])+ "_gtree.csv",id_new_folder)
    id_new_folder3 = create_folder_in_drive(drive, "fairness_data", id_new_folder)
    id_new_folder4 = create_folder_in_drive(drive, "face_data", id_new_folder)
    queries.loc[queries["cardinal"]==x["cardinal"], 'products_folder'] = id_new_folder
    queries.loc[queries["cardinal"]==x["cardinal"], 'products_gtree'] = id_new_file
    queries.loc[queries["cardinal"]==x["cardinal"], 'reviews_folder'] = id_new_folder2
    queries.loc[queries["cardinal"]==x["cardinal"], 'fairness_data'] = id_new_folder3
    queries.loc[queries["cardinal"]==x["cardinal"], 'face_data'] = id_new_folder4
    queries = drop_unnamed(queries)
    queries_csv = queries.to_csv("queries.csv")
    upload_file("queries.csv","1BJCILjUPI5gUIdkgT7nWwU_uz0VZhY7V")

In [ ]:
query=1
products_gtree_id = queries.loc[queries["cardinal"]==query,"products_gtree"].tolist()[0]
products_folder_id = queries.loc[queries["cardinal"]==query,"products_folder"].tolist()[0]
reviews_folder_id = queries.loc[queries["cardinal"]==query,"reviews_folder"].tolist()[0]
fairness_data_id = queries.loc[queries["cardinal"]==query,"fairness_data"].tolist()[0]
face_data_id = queries.loc[queries["cardinal"]==query,"face_data"].tolist()[0]

download = drive.CreateFile({'id': products_gtree_id})
download.GetContentFile(str(query)+'_gtree.csv')

gtree = pd.read_csv(str(query)+'_gtree.csv')
gtree = drop_unnamed(gtree)

In [ ]:
asinList = gtree['asin'].tolist()
asin_count = 0
for asin in asinList:
  asin_count = asin_count+1
  drive = authenticate()

  products_reviews_id = gtree.loc[gtree["asin"]==asin,"reviews_data"].tolist()[0]
  download = drive.CreateFile({'id': products_reviews_id})
  download.GetContentFile(asin+'_reviews.csv')
  reviews_df = pd.read_csv(asin+'_reviews.csv')
  reviews_df = drop_unnamed(reviews_df)

  gfolder_clarifai_id = gtree.loc[gtree["asin"]==asin,"gfolder_clarifai"].tolist()[0]
  
  genderize_df = pd.DataFrame(columns=["user_id","age","gender","ethnicity"])

  print(reviews_df)
  count = 0
  for i,review in reviews_df.iterrows():
    count = count + 1
    user_id=-1
    if not pd.isna(review["profileUrl"]):
      user_id = review["profileUrl"].split("/")[3]
      user = "{'profileUrl':'https://www.amazon.com/gp/profile/" + review["profileUrl"].split("/")[3] +"/ref=cm_cr_getr_d_gw_btm?ie=UTF8','profileImg':'https://www.amazon.com/avatar/default/" + review["profileUrl"].split("/")[3] + "?square=true&max_width=460'}"
    else:
      user = "{}"
      user_id = "missingAccount." + review["name"]

    print(count, user_id)
    minidf_result = detect_gender(user_id, str(review["name"])) 
    genderize_df = genderize_df.append(minidf_result, ignore_index=True)
    
  print(genderize_df)
  genderize_df = drop_unnamed(genderize_df)
  genderize_df.to_csv(str(asin_count) + ' - '+ asin+'.csv')
  upload_file(str(asin_count) + ' - '+ asin+'.csv',"1o_ajBxjV1RO9_DyrDUTiBTJN6RyX9yk_")

In [ ]:
drive = authenticate()
genderize_df = drop_unnamed(genderize_df)
genderize_df.to_csv(str(1) + ' - '+ asin+'.csv')
upload_file(str(1) + ' - '+ asin+'.csv',"1o_ajBxjV1RO9_DyrDUTiBTJN6RyX9yk_")

In [ ]:
asinList = gtree['asin'].tolist()
for asin in asinList:
  drive = authenticate()
  product_position = gtree.loc[gtree["asin"]==asin,"position"].tolist()[0]

  products_reviews_id = gtree.loc[gtree["asin"]==asin,"reviews_data"].tolist()[0]

  download = drive.CreateFile({'id': products_reviews_id})
  download.GetContentFile(asin+'_reviews.csv')

  reviews_df = pd.read_csv(asin+'_reviews.csv')
  reviews_df = drop_unnamed(reviews_df)

  gfolder_clarifai_id = gtree.loc[gtree["asin"]==asin,"gfolder_clarifai"].tolist()[0]
  gfolder_rankings_id = gtree.loc[gtree["asin"]==asin,"gfolder_rankings"].tolist()[0]

  if gfolder_clarifai_id == -1 or gfolder_clarifai_id == '-1' or gfolder_clarifai_id=='-1.0':
    df = pd.DataFrame(columns=["user_id","age","gender","ethnicity"])
    df.to_csv(str(product_position) + ' - '+ asin+'.csv')
    file_id = upload_file(str(product_position) + ' - '+ asin+'.csv',face_data_id)
    gtree.loc[gtree["asin"]==asin,"gfolder_clarifai"]=file_id
    gtree = drop_unnamed(gtree)
    gtree.to_csv(str(query)+'_gtree.csv')
    upload_file(str(query)+'_gtree.csv',products_folder_id)

  if gfolder_rankings_id == -1 or gfolder_rankings_id == '-1' or gfolder_rankings_id=='-1.0':
    print("OK")
    folder_id = create_folder_in_drive(drive,str(product_position) + ' - '+ asin, fairness_data_id)
    gtree.loc[gtree["asin"]==asin,"gfolder_rankings"]=folder_id
    gtree = drop_unnamed(gtree)
    gtree.to_csv(str(query)+'_gtree.csv')
    upload_file(str(query)+'_gtree.csv',products_folder_id)
    
  gfolder_clarifai_id = gtree.loc[gtree["asin"]==asin,"gfolder_clarifai"].tolist()[0]
  gfolder_rankings_id = gtree.loc[gtree["asin"]==asin,"gfolder_rankings"].tolist()[0]

  df = pd.DataFrame(columns=["position","user_id","user","comment","feedback","date","name","country"])
  for i,review in reviews_df.iterrows():
    #user_id
    user_id=-1
    if not pd.isna(review["profileUrl"]):
      user_id = review["profileUrl"].split("/")[3]
      user = "{'profileUrl':'https://www.amazon.com/gp/profile/" + review["profileUrl"].split("/")[3] +"/ref=cm_cr_getr_d_gw_btm?ie=UTF8','profileImg':'https://www.amazon.com/avatar/default/" + review["profileUrl"].split("/")[3] + "?square=true&max_width=460'}"
    else:
      user = "{}"
      user_id = "missingAccount." + review["name"]
    #comment
    if not pd.isna(review["reviewText"]):
      comment = "{'text':'"+review["reviewText"]+"'}"
    else:
      comment = "{}"
    #feedback
    if not pd.isna(review["voteText"]):
      vote = review["voteText"].split(" ")[0]
      if vote == "One":
        vote = 1
      else: vote = int(vote.replace(',', ''))
    else:
      vote = 0
    feedback = "{'counts': {'useful':"+str(vote)+"}}"
    #date & country
    if not pd.isna(review["dateText"]):
      year = review["dateText"].split(" on ")[1].split(", ")[1]
      month = review["dateText"].split(" on ")[1].split(", ")[0].split(" ")[0]
      if month == "January":
        month = "01"
      if month == "February":
        month = "02"
      if month == "March":
        month = "03"
      if month == "April":
        month = "04"
      if month == "May":
        month = "05"
      if month == "June":
        month = "06"
      if month == "July":
        month = "07"
      if month == "August":
        month = "08"
      if month == "September":
        month = "09"
      if month == "October":
        month = "10"
      if month == "November":
        month = "11"
      if month == "December":
        month = "12"
      day = review["dateText"].split(" on ")[1].split(", ")[0].split(" ")[1]
      country = review["dateText"].split(" on ")[0].split("in ")[1]
      date = str(year) + "-" + str(month) + "-" + str(day)
    else:
      date = -1
      country = -1
    
    new_row = {
        "position":i+1,
        "user_id":user_id,
        "user":user,
        "comment":comment,
        "feedback":feedback,
        "date":date,
        "name":review["name"],
        "country":country
    }
    df = df.append(new_row,ignore_index=True)
  df = drop_unnamed(df)
  df.to_csv("ranking_"+asin+".csv")
  upload_file("ranking_"+asin+".csv",gfolder_rankings_id)

  #gender-guesser DATA

  genderize_df = pd.DataFrame(columns=["user_id","age","gender","ethnicity"])
  count = 0
  for i,review in df.iterrows():
    count = count + 1
    print(count, review["user_id"])
    minidf_result = detect_gender(review["user_id"], str(review["name"])) 
    genderize_df = genderize_df.append(minidf_result, ignore_index=True)
    
  genderize_df = drop_unnamed(genderize_df)
  genderize_df.to_csv(str(product_position) + ' - '+ asin+'.csv')
  upload_file(str(product_position) + ' - '+ asin+'.csv',face_data_id)